In [2]:
import torch
import config
import cv2
from utils import (
    get_model_instance_segmentation,
    collate_fn,
    get_transform,
    myOwnDataset,
)

In [3]:
print("Torch version:", torch.__version__)

Torch version: 1.11.0+cu102


In [4]:
# create own Dataset
my_dataset = myOwnDataset(
    root=config.train_data_dir, annotation=config.train_coco, transforms=get_transform()
)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [5]:
print(my_dataset.ids)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [6]:
# own DataLoader
data_loader = torch.utils.data.DataLoader(
    my_dataset,
    batch_size=config.train_batch_size,
    shuffle=config.train_shuffle_dl,
    num_workers=config.num_workers_dl,
    collate_fn=collate_fn,
)

In [18]:
# import numpy as np
# import matplotlib.pyplot as plt

# %matplotlib inline
    
# # obtain one batch of training images
# dataiter = iter(data_loader)
# images, labels = dataiter.next()
# images = images.numpy()

# plot the images in the batch, along with the corresponding labels
# fig = plt.figure(figsize=(25, 4))
# for idx in np.arange(5):
#     ax = fig.add_subplot(2, 5/2, idx+1, xticks=[], yticks=[])
#     ax.imshow(np.squeeze(image[idx]), cmap='gray')
#     ax.set_title(classes[labels[idx]])

In [19]:
# select device (whether GPU or CPU)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [20]:
# DataLoader is iterable over Dataset
for imgs, annotations in data_loader:
    imgs = list(img.to(device) for img in imgs)S
    annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
#     print(annotations)

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'><class 'torch.Tensor'>

<class 'torch.Tensor'>
<class 'torch.Tensor'><class 'torch.Tensor'>

[tensor([[[0.9216, 0.8941, 0.8980,  ..., 0.9765, 0.9961, 1.0000],
         [0.8510, 0.8510, 0.8902,  ..., 0.9608, 0.9686, 0.9725],
         [0.8667, 0.8706, 0.9020,  ..., 0.9686, 0.9608, 0.9529],
         ...,
         [0.8902, 0.8706, 0.8667,  ..., 0.9176, 0.9333, 0.9059],
         [0.8510, 0.8667, 0.9020,  ..., 0.9137, 0.9216, 0.8980],
         [0.8431, 0.8824, 0.9412,  ..., 0.9176, 0.9216, 0.8902]],

        [[0.9216, 0.8941, 0.8980,  ..., 0.9765, 0.9961, 1.0000],
         [0.8510, 0.8510, 0.8902,  ..., 0.9608, 0.9686, 0.9725],
         [0.8667, 0.8706, 0.9020,  ..., 0.9686, 0.9608, 0.9529],
         ...,
         [0.8902, 0.8706, 0.8667,  ..., 0.9176, 0.9333, 0.9059],
         [0.8510, 0.8667, 0.9020,  ..., 0.9137, 0.9216, 0.8980],
         [0.8431, 0.8824

In [ ]:
model = get_model_instance_segmentation(config.num_classes)

# move model to the right device
model.to(device)

In [ ]:
# parameters
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    params, lr=config.lr, momentum=config.momentum, weight_decay=config.weight_decay
)

len_dataloader = len(data_loader)

In [ ]:
for epoch in range(config.num_epochs):
    print(f"Epoch: {epoch}/{config.num_epochs}")
    model.train()
    i = 0
    for imgs, annotations in data_loader:
        i += 1
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        loss_dict = model(imgs, annotations)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        print(f"Iteration: {i}/{len_dataloader}, Loss: {losses}")